In [1]:
# ================================================
#   Data Quality Dimensions – Simple Demonstration
#   Example dataset: customers + purchases
#   Covers: Accuracy, Completeness, Consistency,
#           Timeliness, Validity, Uniqueness
# ================================================

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# --------------------------------------------
# 1) Create a small demo dataset
# --------------------------------------------
data = {
    "customer_id": [101, 102, 102, 103, None],
    "name": ["Ali", "Sara", "Sara", "Omar", "Laila"],
    "age": [30, -5, -5, 140, 28],                    # accuracy/validity issues
    "purchase_amount": [200, 150, np.nan, 300, 250], # completeness issues
    "city": ["Riyadh", "Jeddah", "Jeddah ", "Dammam", "Riyadh"],  # consistency: whitespace
    "purchase_date": [
        "2024-11-01",
        "2021-01-01",
        "2021-01-01",
        "2030-05-01",  # future date: validity/timeliness issue
        None
    ]
}

df = pd.DataFrame(data)
df["purchase_date"] = pd.to_datetime(df["purchase_date"], errors="coerce")

print("=== Sample Dataset ===")
display(df)

# --------------------------------------------
# Helper: current date for timeliness checks
# --------------------------------------------
today = datetime.now()

# --------------------------------------------
# 2) Accuracy Check
#    Example: ages must be between 0 and 120
# --------------------------------------------
accuracy_issues = df[(df["age"] < 0) | (df["age"] > 120)]
print("\n=== Accuracy Issues (Age Outside Reasonable Range) ===")
display(accuracy_issues)

# --------------------------------------------
# 3) Completeness Check
#    Missing values in essential fields
# --------------------------------------------
completeness_issues = df[df.isna().any(axis=1)]
print("\n=== Completeness Issues (Missing Values) ===")
display(completeness_issues)

# --------------------------------------------
# 4) Consistency Check
#    City names containing extra spaces
# --------------------------------------------
df["city_clean"] = df["city"].str.strip()
consistency_issues = df[df["city"] != df["city_clean"]]
print("\n=== Consistency Issues (Whitespace Inconsistency) ===")
display(consistency_issues)

# --------------------------------------------
# 5) Timeliness Check
#    Example: purchase date must be within last 2 years
# --------------------------------------------
two_years_ago = today - timedelta(days=730)
timeliness_issues = df[(df["purchase_date"] < two_years_ago) |
                       (df["purchase_date"].isna())]
print("\n=== Timeliness Issues (Old or Missing Dates) ===")
display(timeliness_issues)

# --------------------------------------------
# 6) Validity Check
#    Purchase date cannot be in the future
# --------------------------------------------
validity_issues = df[df["purchase_date"] > today]
print("\n=== Validity Issues (Dates in the Future) ===")
display(validity_issues)

# --------------------------------------------
# 7) Uniqueness Check
#    Duplicate customer records
# --------------------------------------------
duplicate_records = df[df.duplicated(subset=["customer_id", "name"], keep=False)]
print("\n=== Uniqueness Issues (Duplicate Customer Records) ===")
display(duplicate_records)


=== Sample Dataset ===


,customer_id,name,age,purchase_amount,city,purchase_date
0,101.0,Ali,30,200.0,Riyadh,2024-11-01
1,102.0,Sara,-5,150.0,Jeddah,2021-01-01
2,102.0,Sara,-5,NaN,Jeddah,2021-01-01
3,103.0,Omar,140,300.0,Dammam,2030-05-01
4,NaN,Laila,28,250.0,Riyadh,NaT



=== Accuracy Issues (Age Outside Reasonable Range) ===


,customer_id,name,age,purchase_amount,city,purchase_date
1,102.0,Sara,-5,150.0,Jeddah,2021-01-01
2,102.0,Sara,-5,NaN,Jeddah,2021-01-01
3,103.0,Omar,140,300.0,Dammam,2030-05-01



=== Completeness Issues (Missing Values) ===


...

,customer_id,name,age,purchase_amount,city,purchase_date,city_clean
1,102.0,Sara,-5,150.0,Jeddah,2021-01-01,Jeddah
2,102.0,Sara,-5,NaN,Jeddah,2021-01-01,Jeddah
4,NaN,Laila,28,250.0,Riyadh,NaT,Riyadh



=== Validity Issues (Dates in the Future) ===


,customer_id,name,age,purchase_amount,city,purchase_date,city_clean
3,103.0,Omar,140,300.0,Dammam,2030-05-01,Dammam



=== Uniqueness Issues (Duplicate Customer Records) ===


,customer_id,name,age,purchase_amount,city,purchase_date,city_clean
1,102.0,Sara,-5,150.0,Jeddah,2021-01-01,Jeddah
2,102.0,Sara,-5,NaN,Jeddah,2021-01-01,Jeddah
